## Introduction

This notebook is created to understand the techniques of data wrangling.

The dataset used here is the **'default of credit card clients Data Set'** from UCI, available **[here](https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients)**

In [40]:
# Section 1: Data Acquisition

# We simply load the required libraries and the datasets

# URL of the dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls'

# Importing the required libraries

import pandas as pd

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style('darkgrid')


# Importing the original dataset and saving it as 'raw_df'
raw_df = pd.read_excel(url,header=1)

# Creating a copy of the original dataset to work with
df = raw_df.copy()

In [41]:
# Eyeballing the first 5 rows of the dataset to make sure the data is loaded properly
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [42]:
# Now we explore the datset by calling the .info() method on the dataframe

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   ID                          30000 non-null  int64
 1   LIMIT_BAL                   30000 non-null  int64
 2   SEX                         30000 non-null  int64
 3   EDUCATION                   30000 non-null  int64
 4   MARRIAGE                    30000 non-null  int64
 5   AGE                         30000 non-null  int64
 6   PAY_0                       30000 non-null  int64
 7   PAY_2                       30000 non-null  int64
 8   PAY_3                       30000 non-null  int64
 9   PAY_4                       30000 non-null  int64
 10  PAY_5                       30000 non-null  int64
 11  PAY_6                       30000 non-null  int64
 12  BILL_AMT1                   30000 non-null  int64
 13  BILL_AMT2                   30000 non-null  int64
 14  BILL_A

**Data Definitions**

This research employed a binary variable, default payment (Yes = 1, No = 0), as the response variable. This study reviewed the literature and used the following 23 variables as explanatory variables:

**LIMIT_BAL:** Amount of the given credit (NT dollar): it includes both the individual consumer credit and his/her family (supplementary) credit.

**SEX:** Gender (1 = male; 2 = female).

**EDUCATION:** Education (1 = graduate school; 2 = university; 3 = high school; 4 = others).

**MARRIAGE:** Marital status (1 = married; 2 = single; 3 = others).

**AGE:** Age (year).

**PAY_0,PAY_2 - PAY_6:** History of past payment. We tracked the past monthly payment records (from April to September, 2005) as follows: **PAY_0:** the repayment status in September, 2005; **PAY_1:** = the repayment status in August, 2005; . . .;**PAY_6:** = the repayment status in April, 2005. The measurement scale for the repayment status is: -1 = pay duly; 1 = payment delay for one month; 2 = payment delay for two months; . . .; 8 = payment delay for eight months; 9 = payment delay for nine months and above.

**BILL_AMT1-BILL_AMT6:** Amount of bill statement (NT dollar). **BILL_AMT1** = amount of bill statement in September, 2005; **BILL_AMT2** = amount of bill statement in August, 2005; . . .; **BILL_AMT6** = amount of bill statement in April, 2005.

**PAY_AMT1-PAY_AMT6:** Amount of previous payment (NT dollar). **PAY_AMT1** = amount paid in September, 2005; **PAY_AMT2** = amount paid in August, 2005; . . .; **PAY_AMT6**: amount paid in April, 2005. 

At the outset it seems that there are no missing values. But on digging deeper into the data definitions available from the dataset provider, the columns Pay_0 to Pay_6 are supposed to be in either -1/1/2...9. The documentation does not have a definition for '0' and hence we will take '0' or '-2' values for these fields. Need to investigate further

In [43]:
# Exploring the dependent feature 'default payment next month'

# Changing the name to 'default'

# Also changing the name of the column of PAY_0 to PAY1, for alignment purpose

df.rename(columns = {'default payment next month':'default','PAY_0':'PAY_1'},inplace=True)
df['default'].value_counts(normalize=True)

0    0.7788
1    0.2212
Name: default, dtype: float64

We have a biased dataset with almost 80% not defaulting on payments the next month

In [44]:
# Exploring the dataset in detail using the describe() method
df.describe().T

,count,mean,std,min,25%,50%,75%,max
ID,30000.0,15000.500000,8660.398374,1.0,7500.75,15000.5,22500.25,30000.0
LIMIT_BAL,30000.0,167484.322667,129747.661567,10000.0,50000.00,140000.0,240000.00,1000000.0
SEX,30000.0,1.603733,0.489129,1.0,1.00,2.0,2.00,2.0
EDUCATION,30000.0,1.853133,0.790349,0.0,1.00,2.0,2.00,6.0
MARRIAGE,30000.0,1.551867,0.521970,0.0,1.00,2.0,2.00,3.0
AGE,30000.0,35.485500,9.217904,21.0,28.00,34.0,41.00,79.0
PAY_1,30000.0,-0.016700,1.123802,-2.0,-1.00,0.0,0.00,8.0
PAY_2,30000.0,-0.133767,1.197186,-2.0,-1.00,0.0,0.00,8.0
PAY_3,30000.0,-0.166200,1.196868,-2.0,-1.00,0.0,0.00,8.0
PAY_4,30000.0,-0.220667,1.169139,-2.0,-1.00,0.0,0.00,8.0


Now let us start exploring each feature, to understand them better and also identify in case of any miss-classifications

In [45]:
# Feature 1 : LIMIT BAL

df['LIMIT_BAL'].describe()

count      30000.000000
mean      167484.322667
std       129747.661567
min        10000.000000
25%        50000.000000
50%       140000.000000
75%       240000.000000
max      1000000.000000
Name: LIMIT_BAL, dtype: float64

Range is pretty wide. From 10K to a 1M. But nothing of concern here to clean

In [47]:
# Feature 2: SEX

df['SEX'].value_counts(normalize=True)

2    0.603733
1    0.396267
Name: SEX, dtype: float64

We have a higher proportion of female users compared to male.

In [48]:
# Feature 3: MARRIAGE

df['MARRIAGE'].value_counts()

2    15964
1    13659
3      323
0       54
Name: MARRIAGE, dtype: int64

In [59]:
df.loc[df['MARRIAGE']==0,'MARRIAGE'] = 3
df['MARRIAGE'].value_counts()

2    15964
1    13659
3      377
Name: MARRIAGE, dtype: int64

In [60]:
# Feature 4: AGE

df['AGE'].describe()

count    30000.000000
mean        35.485500
std          9.217904
min         21.000000
25%         28.000000
50%         34.000000
75%         41.000000
max         79.000000
Name: AGE, dtype: float64

Seems to be no issues as such. Ranging from 21 to 79. Well within a reasonable min and max range

In [63]:
# Feature 5: PAY_n variables Let us replace the undefined values. As the logical labeling/
# would be 0,1,2,3.... We will replace -1, -2 as '0'

fill = (df.PAY_1 == -2) | (df.PAY_1 == -1) | (df.PAY_1 ==0)
df.loc[fill,'PAY_1'] = 0

fill = (df.PAY_2 == -2) | (df.PAY_2 == -1) | (df.PAY_2 ==0)
df.loc[fill,'PAY_2'] = 0

fill = (df.PAY_3 == -2) | (df.PAY_3 == -1) | (df.PAY_3 ==0)
df.loc[fill,'PAY_3'] = 0

fill = (df.PAY_4 == -2) | (df.PAY_4 == -1) | (df.PAY_4 ==0)
df.loc[fill,'PAY_4'] = 0

fill = (df.PAY_5 == -2) | (df.PAY_5 == -1) | (df.PAY_5 ==0)
df.loc[fill,'PAY_5'] = 0

fill = (df.PAY_6 == -2) | (df.PAY_6 == -1) | (df.PAY_6 ==0)
df.loc[fill,'PAY_6'] = 0

The remaining features i.e. BILL_AMT_n and PAY_AMT_n seem okay, based on the initial .describe() method

So now we have cleaned our dataset of any missing/miss classified points. 